<a href="https://colab.research.google.com/github/Ed-spo/Agente_IA_Ex/blob/main/Agentes_IA_Imersao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00


A célula `SIx2Bhm1rxgm` importa as bibliotecas necessárias e configura a chave de API para usar o modelo Gemini do Google com LangChain.

*   **`from google.colab import userdata`**: Importa a função `userdata` do módulo `google.colab` para acessar segredos armazenados no Colab, como a chave de API.
*   **`from langchain_google_genai import ChatGoogleGenerativeAI`**: Importa a classe `ChatGoogleGenerativeAI` do módulo `langchain_google_genai`. Esta classe é usada para interagir com os modelos de chat do Google Generative AI através do framework LangChain.
*   **`GOOGLE_API_KEY = userdata.get('GOOGLE_GEMINI_API_KEY')`**: Recupera a chave de API armazenada como um segredo no Colab com o nome 'GOOGLE_GEMINI_API_KEY' e a atribui à variável `GOOGLE_API_KEY`.

In [3]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GOOGLE_GEMINI_API_KEY')


In [4]:
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash',
    temperature=0, #! Indica a criatividade do modelo
    api_key=GOOGLE_API_KEY,
    max_output_tokens=1024,

)

In [6]:
resp_test = llm.invoke('Qual é a captal do Brasil? ')
print(resp_test.content)

A capital do Brasil é **Brasília**.


In [7]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [8]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

# Criando uma classe
class TriagemOut(BaseModel):
    decisao: Literal['AUTO_RESOLVER', 'PEDIR_INFO', 'ABRIR_CHAMADO']
    urgencia: Literal['BAIXA', 'MEDIA', 'ALTA']
    campos_faltantes: List[str] = Field(default_factory=list)


In [9]:
llm_triagem = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash',
    temperature=0, #! Indica a criatividade do modelo
    api_key=GOOGLE_API_KEY,
    max_output_tokens=1024
)

In [14]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
  saida: TriagemOut = triagem_chain.invoke([
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=mensagem)
  ])
  return saida.model_dump()


In [15]:
testes = ['Posso reembolsar a internet?',
          'Quero mais 5 dias de trabalho remoto. Como faço?',
          'Posso reembolsar cursos ou treinamentos da Alura?',
          'Quantos mundiais tem o Palmeiras?']

In [16]:
for msg_teste in testes:
  print(f'Pergunta: {msg_teste}\n')
  print(f'Resposta: {triagem(msg_teste)}\n')

Pergunta: Posso reembolsar a internet?

Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?

Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos ou treinamentos da Alura?

Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantos mundiais tem o Palmeiras?

Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

